### Xgboost with optuna

## 📌 Graduates Admission Prediction — Project Overview

## 🧩 Problem Statement
The purpose of this project is to build an **XGBoost Regressor** model that can predict the probability, or **Chance of Admit (COA)**, for a student applying to a Master's program.
The prediction is based on the student's academic profile and background.

---

## 📊 Dataset Information

## 🏷️ Dataset Name
Graduate Admissions Dataset

## 🌐 Source
Kaggle

## 🔗 URL
`https://www.kaggle.com/datasets/mukeshmanral/graduates-admission-prediction`

---

## 📐 Dataset Shape
- **Total Rows:** 500
- **Total Columns:** 9

---

## 🧬 Dataset Columns
- **Serial No.** – Student serial number (typically dropped)
- **GRE Score** – Graduate Record Examinations score (out of 340)
- **TOEFL Score** – Test of English as a Foreign Language score (out of 120)
- **University Rating** – University rating (out of 5)
- **SOP** – Strength of Statement of Purpose (out of 5)
- **LOR** – Strength of Letter of Recommendation (out of 5)
- **CGPA** – Undergraduate GPA/Cumulative Grade Point Average (out of 10)
- **Research** – Research experience (0 = No, 1 = Yes)
- **Chance of Admit** – Target probability of admission (ranging from 0 to 1)

---

## 🎯 Features and Target Used in This Project

## 🔹 Features (X) — 8 Total
We will use the following 8 features:

- GRE Score
- TOEFL Score
- University Rating
- SOP
- LOR
- CGPA
- Research
- Age *(Note: The original dataset may not contain 'Age' directly; this list assumes using the standard features and potentially including 'Age' if derived or available.)*

## 🎯 Target (y)
- **Chance of Admit** (Continuous value from 0 to 1)

---

## 📚 Steps We Will Perform

1. Load the dataset
2. Explore and Preprocess the dataset (Handle potential missing values/outliers, and drop 'Serial No.')
3. Select features (X) and target (y)
4. Split the data into training and testing sets
5. **Train the initial XGBoost Regressor model**
6. **Find the Best Parameter using Optuna** (Hyperparameter optimization for XGBoost)
7. Build the final optimized model
8. Evaluate the model performance (using metrics like RMSE, $R^2$)

### Load the specific library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna
from ydata_profiling import ProfileReport
from sklearn.metrics import mean_squared_error

### Load the dataset

In [2]:
df=pd.read_csv("C:\\machile learnig implementation\\Supervised\\linear regression\\Admission_prediction\\Admission_Prediction.csv")

In [3]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332,108,5,4.5,4.0,9.02,1,0.87
496,497,337,117,5,5.0,5.0,9.87,1,0.96
497,498,330,120,5,4.5,5.0,9.56,1,0.93
498,499,312,103,4,4.0,5.0,8.43,0,0.73


### select the label and feature column

In [4]:
df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [5]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [6]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [ ]:
## this is the label column
x=df.drop(columns=["Serial No.","Chance of Admit "])

In [8]:
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1
496,337,117,5,5.0,5.0,9.87,1
497,330,120,5,4.5,5.0,9.56,1
498,312,103,4,4.0,5.0,8.43,0


In [9]:
## this is the label columns
y=df["Chance of Admit "]

In [10]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit , Length: 500, dtype: float64

In [11]:
from sklearn.preprocessing import StandardScaler
scaller=StandardScaler()
x=scaller.fit_transform(x)

In [12]:
x

array([[ 1.81923762,  1.77886545,  0.77558214, ...,  1.09894429,
         1.77680627,  0.88640526],
       [ 0.66714832, -0.03160087,  0.77558214, ...,  1.09894429,
         0.48585943,  0.88640526],
       [-0.0418297 , -0.52536441, -0.09979274, ...,  0.01730621,
        -0.95404281,  0.88640526],
       ...,
       [ 1.19888185,  2.10804114,  1.65095702, ...,  1.63976333,
         1.62785086,  0.88640526],
       [-0.39631872, -0.68995225,  0.77558214, ...,  1.63976333,
        -0.24236699, -1.12815215],
       [ 0.93301508,  0.95592621,  0.77558214, ...,  1.09894429,
         0.76721964, -1.12815215]])

## optuna

In [13]:
def objective(trail,data=x,target=y):
    train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.20,random_state=30)
    param={
        "tree_method": "hist" ,
        "lambda":trail.suggest_float("lambda",1e-4,10.0, log=True),
        "alpha":trail.suggest_float("alpha",1e-4,10.0, log=True),
        "colsample_bytree":trail.suggest_categorical("colsample_bytree",[0.1,0.2,0.4,0.5,0.6,0.7,0.8,0.9,1]),
        "subsample":trail.suggest_categorical("subsample",[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
        "learning_rate":trail.suggest_categorical("learning_rate",[0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,1]),
       "n_estimators": 3000,
        "max_depth":trail.suggest_categorical("max_depth",[3,4,5,6,7,8,9,10,11,12]),
        "random_state":trail.suggest_categorical("random_state",[10,20,50,200,1800,370023456]),
        "min_child_weight":trail.suggest_int("min_child_weight",1,200)
        #"versbosity":3
    }
    xgb_reg_model=xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y, pred_xgb)
    return rmse

In [14]:
find_param=optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials=10)
find_param.best_params

[I 2025-12-16 05:57:59,763] A new study created in memory with name: no-name-2d0edf1f-fac0-40b4-8fbb-0c578301cb55


[0]	validation_0-rmse:0.12446
[1]	validation_0-rmse:0.11176
[2]	validation_0-rmse:0.09924
[3]	validation_0-rmse:0.09054
[4]	validation_0-rmse:0.08448
[5]	validation_0-rmse:0.07972
[6]	validation_0-rmse:0.07653
[7]	validation_0-rmse:0.07380
[8]	validation_0-rmse:0.07203
[9]	validation_0-rmse:0.07101
[10]	validation_0-rmse:0.07018
[11]	validation_0-rmse:0.07001
[12]	validation_0-rmse:0.06991
[13]	validation_0-rmse:0.06945
[14]	validation_0-rmse:0.06978
[15]	validation_0-rmse:0.06959
[16]	validation_0-rmse:0.07016
[17]	validation_0-rmse:0.07037
[18]	validation_0-rmse:0.07012
[19]	validation_0-rmse:0.07085
[20]	validation_0-rmse:0.07066
[21]	validation_0-rmse:0.07065
[22]	validation_0-rmse:0.07065
[23]	validation_0-rmse:0.07037
[24]	validation_0-rmse:0.07084
[25]	validation_0-rmse:0.07079
[26]	validation_0-rmse:0.07116
[27]	validation_0-rmse:0.07135
[28]	validation_0-rmse:0.07155
[29]	validation_0-rmse:0.07240
[30]	validation_0-rmse:0.07290
[31]	validation_0-rmse:0.07326
[32]	validation_0-

[I 2025-12-16 05:58:05,616] Trial 0 finished with value: 0.00796374281743776 and parameters: {'lambda': 0.00013918969643825414, 'alpha': 0.005296195015845637, 'colsample_bytree': 0.7, 'subsample': 0.9, 'learning_rate': 0.2, 'max_depth': 11, 'random_state': 1800, 'min_child_weight': 11}. Best is trial 0 with value: 0.00796374281743776.


[0]	validation_0-rmse:0.14504
[1]	validation_0-rmse:0.14467
[2]	validation_0-rmse:0.14397
[3]	validation_0-rmse:0.14338
[4]	validation_0-rmse:0.14300
[5]	validation_0-rmse:0.14267
[6]	validation_0-rmse:0.14198
[7]	validation_0-rmse:0.14156
[8]	validation_0-rmse:0.14120
[9]	validation_0-rmse:0.14066
[10]	validation_0-rmse:0.14018
[11]	validation_0-rmse:0.13946
[12]	validation_0-rmse:0.13912
[13]	validation_0-rmse:0.13869
[14]	validation_0-rmse:0.13838
[15]	validation_0-rmse:0.13787
[16]	validation_0-rmse:0.13743
[17]	validation_0-rmse:0.13721
[18]	validation_0-rmse:0.13699
[19]	validation_0-rmse:0.13667
[20]	validation_0-rmse:0.13595
[21]	validation_0-rmse:0.13559
[22]	validation_0-rmse:0.13498
[23]	validation_0-rmse:0.13462
[24]	validation_0-rmse:0.13421
[25]	validation_0-rmse:0.13352
[26]	validation_0-rmse:0.13317
[27]	validation_0-rmse:0.13274
[28]	validation_0-rmse:0.13203
[29]	validation_0-rmse:0.13168
[30]	validation_0-rmse:0.13108
[31]	validation_0-rmse:0.13064
[32]	validation_0-

[I 2025-12-16 05:58:11,847] Trial 1 finished with value: 0.004523794554627408 and parameters: {'lambda': 0.005563284311412617, 'alpha': 0.5318626988882879, 'colsample_bytree': 0.2, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 370023456, 'min_child_weight': 5}. Best is trial 1 with value: 0.004523794554627408.


...

[0]	validation_0-rmse:0.14572
[1]	validation_0-rmse:0.14572
[2]	validation_0-rmse:0.14573
[3]	validation_0-rmse:0.14572
[4]	validation_0-rmse:0.14572
[5]	validation_0-rmse:0.14527
[6]	validation_0-rmse:0.14452
[7]	validation_0-rmse:0.14453
[8]	validation_0-rmse:0.14453
[9]	validation_0-rmse:0.14371
[10]	validation_0-rmse:0.14368
[11]	validation_0-rmse:0.14367
[12]	validation_0-rmse:0.14365
[13]	validation_0-rmse:0.14322
[14]	validation_0-rmse:0.14319
[15]	validation_0-rmse:0.14263
[16]	validation_0-rmse:0.14262
[17]	validation_0-rmse:0.14266
[18]	validation_0-rmse:0.14267
[19]	validation_0-rmse:0.14220
[20]	validation_0-rmse:0.14159
[21]	validation_0-rmse:0.14117
[22]	validation_0-rmse:0.14117
[23]	validation_0-rmse:0.14117
[24]	validation_0-rmse:0.14117
[25]	validation_0-rmse:0.14083
[26]	validation_0-rmse:0.14021
[27]	validation_0-rmse:0.13967
[28]	validation_0-rmse:0.13970
[29]	validation_0-rmse:0.13926
[30]	validation_0-rmse:0.13926
[31]	validation_0-rmse:0.13875
[32]	validation_0-

[I 2025-12-16 05:58:56,179] Trial 8 finished with value: 0.006169107415663454 and parameters: {'lambda': 2.038524741605559, 'alpha': 0.23071102195113694, 'colsample_bytree': 0.5, 'subsample': 0.2, 'learning_rate': 0.01, 'max_depth': 10, 'random_state': 1800, 'min_child_weight': 40}. Best is trial 1 with value: 0.004523794554627408.


[0]	validation_0-rmse:0.14188
[1]	validation_0-rmse:0.13839
[2]	validation_0-rmse:0.13525
[3]	validation_0-rmse:0.13176
[4]	validation_0-rmse:0.12923
[5]	validation_0-rmse:0.12658
[6]	validation_0-rmse:0.12393
[7]	validation_0-rmse:0.12400
[8]	validation_0-rmse:0.12394
[9]	validation_0-rmse:0.12149
[10]	validation_0-rmse:0.11952
[11]	validation_0-rmse:0.11762
[12]	validation_0-rmse:0.11575
[13]	validation_0-rmse:0.11382
[14]	validation_0-rmse:0.11207
[15]	validation_0-rmse:0.11031
[16]	validation_0-rmse:0.11027
[17]	validation_0-rmse:0.10890
[18]	validation_0-rmse:0.10889
[19]	validation_0-rmse:0.10741
[20]	validation_0-rmse:0.10584
[21]	validation_0-rmse:0.10445
[22]	validation_0-rmse:0.10442
[23]	validation_0-rmse:0.10437
[24]	validation_0-rmse:0.10324
[25]	validation_0-rmse:0.10218
[26]	validation_0-rmse:0.10101
[27]	validation_0-rmse:0.09999
[28]	validation_0-rmse:0.10001
[29]	validation_0-rmse:0.09922
[30]	validation_0-rmse:0.09832
[31]	validation_0-rmse:0.09743
[32]	validation_0-

[I 2025-12-16 05:59:01,861] Trial 9 finished with value: 0.006505038526677962 and parameters: {'lambda': 0.001987569763725617, 'alpha': 0.0017780598413021564, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.05, 'max_depth': 10, 'random_state': 1800, 'min_child_weight': 96}. Best is trial 1 with value: 0.004523794554627408.


{'lambda': 0.005563284311412617,
 'alpha': 0.5318626988882879,
 'colsample_bytree': 0.2,
 'subsample': 0.4,
 'learning_rate': 0.01,
 'max_depth': 9,
 'random_state': 370023456,
 'min_child_weight': 5}

In [16]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.007964,2025-12-16 05:57:59.764623,2025-12-16 05:58:05.616890,0 days 00:00:05.852267,0.005296,0.7,0.000139,0.20,11,11,1800,0.9,COMPLETE
1,1,0.004524,2025-12-16 05:58:05.617890,2025-12-16 05:58:11.847646,0 days 00:00:06.229756,0.531863,0.2,0.005563,0.01,9,5,370023456,0.4,COMPLETE
2,2,0.021256,2025-12-16 05:58:11.848646,2025-12-16 05:58:18.476137,0 days 00:00:06.627491,4.981158,0.9,0.021787,0.10,7,157,1800,0.4,COMPLETE
3,3,0.021309,2025-12-16 05:58:18.476137,2025-12-16 05:58:25.029828,0 days 00:00:06.553691,0.001546,0.5,0.736325,0.05,7,181,10,0.6,COMPLETE
4,4,0.005047,2025-12-16 05:58:25.031835,2025-12-16 05:58:32.069824,0 days 00:00:07.037989,0.000422,0.1,0.157246,0.30,12,71,1800,0.7,COMPLETE
5,5,0.017137,2025-12-16 05:58:32.071828,2025-12-16 05:58:38.933600,0 days 00:00:06.861772,5.347620,0.5,0.000618,0.05,3,133,10,0.6,COMPLETE
6,6,0.006035,2025-12-16 05:58:38.933600,2025-12-16 05:58:45.135639,0 days 00:00:06.202039,0.075653,0.2,0.005205,0.30,8,12,20,0.7,COMPLETE
7,7,0.007039,2025-12-16 05:58:45.136639,2025-12-16 05:58:50.515625,0 days 00:00:05.378986,0.040838,0.9,2.475913,0.05,5,170,200,0.9,COMPLETE
8,8,0.006169,2025-12-16 05:58:50.517625,2025-12-16 05:58:56.179039,0 days 00:00:05.661414,0.230711,0.5,2.038525,0.01,10,40,1800,0.2,COMPLETE
9,9,0.006505,2025-12-16 05:58:56.181044,2025-12-16 05:59:01.861051,0 days 00:00:05.680007,0.001778,0.8,0.001988,0.05,10,96,1800,0.5,COMPLETE


In [18]:
optuna.visualization.plot_optimization_history(find_param)

In [20]:
optuna.visualization.plot_slice(find_param)

In [24]:
optuna.visualization.plot_contour(find_param,["alpha","lambda"])

### my mai work here is find the best parameter so now build the model

### Best parameter is :-

In [25]:
"""
'lambda': 0.005563284311412617,
 'alpha': 0.5318626988882879,
 'colsample_bytree': 0.2,
 'subsample': 0.4,
 'learning_rate': 0.01,
 'max_depth': 9,
 'random_state': 370023456,
 'min_child_weight': 5

 """

"\n'lambda': 0.005563284311412617,\n 'alpha': 0.5318626988882879,\n 'colsample_bytree': 0.2,\n 'subsample': 0.4,\n 'learning_rate': 0.01,\n 'max_depth': 9,\n 'random_state': 370023456,\n 'min_child_weight': 5\n\n "

In [26]:
best_param={
 'lambda': 0.005563284311412617,
 'alpha': 0.5318626988882879,
 'colsample_bytree': 0.2,
 'subsample': 0.4,
 'learning_rate': 0.01,
 'max_depth': 9,
 'random_state': 370023456,
 'min_child_weight': 5
}

In [27]:
xgb_final_model=xgb.XGBRegressor(**best_param)

In [28]:
xgb_final_model.fit(x,y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.2
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [29]:
xgb_final_model.score(x,y)

0.5374704184618166

In [30]:
xgb_model_without_best_param=xgb.XGBRegressor()
xgb_model_without_best_param.fit(x,y)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [31]:
xgb_model_without_best_param.score(x,y)

0.9996626283279645